In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import pandas as pd
import numpy as np
import math 

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from operator import add

from pathlib import Path
home = str(Path.home())

In [50]:
df = pd.read_csv(f"{home}/Project/UserCarData.csv",index_col=0)
df.head()

,name,year,selling_price,km_driven,Region,State or Province,City,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,sold
Sales_ID,,,,,,,,,,,,,,,,,
1,Maruti,2014,450000,145500,East,District of Columbia,Washington,Diesel,Individual,Manual,First_Owner,23.40,1248,74.00,190Nm@ 2000rpm,5,Y
2,Skoda,2014,370000,120000,East,New York,New York City,Diesel,Individual,Manual,Second_Owner,21.14,1498,103.52,250Nm@ 1500-2500rpm,5,Y
3,Honda,2006,158000,140000,Central,Illinois,Chicago,Petrol,Individual,Manual,Third_Owner,17.70,1497,78.00,"12.7@ 2,700(kgm@ rpm)",5,Y
4,Hyundai,2010,225000,127000,Central,Illinois,Chicago,Diesel,Individual,Manual,First_Owner,23.00,1396,90.00,22.4 kgm at 1750-2750rpm,5,Y
5,Maruti,2007,130000,120000,East,New York,New York City,Petrol,Individual,Manual,First_Owner,16.10,1298,88.20,"11.5@ 4,500(kgm@ rpm)",5,Y


In [51]:
features = ['name','year','selling_price','km_driven','Region','State or Province','City','fuel','seller_type','owner','mileage','engine','max_power']
predictors = df[features].dropna()
predictors = pd.get_dummies(predictors)
means = predictors.mean()
sds = predictors.std()
predictors = predictors.apply(lambda x: (x-means)/sds,axis=1)
predictors

,year,selling_price,km_driven,mileage,engine,max_power,name_Ambassador,name_Ashok,name_Audi,name_BMW,...,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark_Dealer,owner_First_Owner,owner_Fourth_Above_Owner,owner_Second_Owner,owner_Test_Drive_Car,owner_Third_Owner
Sales_ID,,,,,,,,,,,,,,,,,,,,,
1,0.004158,-0.245597,1.343692,0.986095,-0.418162,-0.491993,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,-0.895797,-0.403482,0.452334,-0.175401,0.718294,-0.143712,-0.585006,-0.025155,-0.262579
2,0.004158,-0.343928,0.894687,0.426171,0.077975,0.333806,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,-0.895797,-0.403482,0.452334,-0.175401,-1.392012,-0.143712,1.709169,-0.025155,-0.262579
3,-2.066399,-0.604504,1.246848,-0.426102,0.075991,-0.380096,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,1.116183,-0.403482,0.452334,-0.175401,-1.392012,-0.143712,-0.585006,-0.025155,3.807903
4,-1.031121,-0.522152,1.017943,0.886993,-0.124449,-0.044406,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,-0.895797,-0.403482,0.452334,-0.175401,0.718294,-0.143712,-0.585006,-0.025155,-0.262579
5,-1.807579,-0.638919,0.894687,-0.822509,-0.318934,-0.094759,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,1.116183,-0.403482,0.452334,-0.175401,0.718294,-0.143712,-0.585006,-0.025155,-0.262579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8124,-0.254662,-0.405384,0.718607,-0.227899,-0.519374,-0.244421,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,1.116183,-0.403482,0.452334,-0.175401,0.718294,-0.143712,-0.585006,-0.025155,-0.262579
8125,-1.807579,-0.632774,0.877079,-0.649081,0.068052,0.515079,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,-0.895797,-0.403482,0.452334,-0.175401,-1.392012,6.957469,-0.585006,-0.025155,-0.262579
8126,-1.289940,-0.329178,0.894687,-0.029696,-0.418162,-0.494790,-0.022497,-0.011247,-0.071306,-0.123084,...,-0.066679,-0.895797,-0.403482,0.452334,-0.175401,0.718294,-0.143712,-0.585006,-0.025155,-0.262579


In [52]:
target = df['transmission']
target

Sales_ID
1       Manual
2       Manual
3       Manual
4       Manual
5       Manual
         ...  
8124    Manual
8125    Manual
8126    Manual
8127    Manual
8128    Manual
Name: transmission, Length: 7906, dtype: object

In [53]:
X = predictors.values
y = target.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [54]:
def evaluate_baseline(X,y):
    accuracy = {}
    X = pd.Series(X)
    y = pd.Series(y) 
    accuracy_test = None
    accuracy_train = None
    
    if (X.value_counts()[0] > X.value_counts()[1]):
        accuracy_test = (X.value_counts()[0]) / len(X)
    else:
        accuracy_test = (X.value_counts()[1]) / len(X)
        
    if (y.value_counts()[0] > y.value_counts()[1]):
        accuracy_train = (y.value_counts()[0]) / len(y)
    else:
        accuracy_train = (y.value_counts()[1]) / len(y)    

    accuracy['testing_acc'] = accuracy_test
    accuracy['training_acc'] = accuracy_train
    
    return accuracy

In [55]:
evaluate_baseline(y_train,y_test)

{'testing_acc': 0.8662811709432598, 'training_acc': 0.8731028667790893}

In [56]:
mlp = MLPClassifier(hidden_layer_sizes=(1412,), activation='relu', solver='adam', max_iter=10)
mlp.fit(X_train,y_train)
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

/opt/tljh/user/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [57]:
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[ 644   96]
 [  34 4760]]
              precision    recall  f1-score   support

   Automatic       0.95      0.87      0.91       740
      Manual       0.98      0.99      0.99      4794

    accuracy                           0.98      5534
   macro avg       0.97      0.93      0.95      5534
weighted avg       0.98      0.98      0.98      5534



In [58]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[ 182  119]
 [  72 1999]]
              precision    recall  f1-score   support

   Automatic       0.72      0.60      0.66       301
      Manual       0.94      0.97      0.95      2071

    accuracy                           0.92      2372
   macro avg       0.83      0.78      0.81      2372
weighted avg       0.91      0.92      0.92      2372



In [59]:
def activation(net):
    return 1/(1+np.exp(-net))

def train(X,t,nepochs=200,n=0.5,test_size=0.3,val_size=0.3,seed=0):
    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=test_size,random_state=seed)
    X_train2, X_val, t_train2, t_val = train_test_split(X_train, t_train, test_size=val_size,random_state=seed)

    train_accuracy = []
    val_accuracy = []
    nfeatures = X.shape[1]
    np.random.seed(seed)
    w = 2*np.random.uniform(size=(nfeatures,)) - 1
    
    for epoch in range(nepochs):
        y_train2 = X_train2.apply(lambda x: activation(np.dot(w,x)),axis=1)
        y_val = X_val.apply(lambda x: activation(np.dot(w,x)),axis=1)

        train_accuracy.append(sum(t_train2 == np.round(y_train2))/len(t_train2))
        val_accuracy.append(sum(t_val == np.round(y_val))/len(t_val))
                
        for j in range(len(w)):
            w[j] -= n*np.dot((y_train2 - t_train2)*y_train2*(1-y_train2),X_train2.iloc[:,j])
            
    results = pd.DataFrame({"epoch": np.arange(nepochs)+1, 'train_accuracy':train_accuracy,'val_accuracy':val_accuracy,
                            "n":n,'test_size':test_size,'val_size':val_size,'seed':seed
                           }).set_index(['n','test_size','val_size','seed'])
    return w,X_test,t_test,results

def predict(w,X,threshold=0.5):
    y = None
    y = X.apply(lambda x: activation(np.dot(w,x)),axis=1)
    y = y.apply(lambda x: 1 if x >= threshold else 0)  
    return y

In [60]:
target = target.replace('Manual', 0)
target = target.replace('Automatic', 1)

In [61]:
w,X_test,t_test,results = train(predictors,target,seed=0)
y_test = predict(w,X_test)
y_test

Sales_ID
4049    1
4832    1
5124    0
6506    0
47      0
       ..
4192    1
4047    1
4731    0
2009    0
602     0
Length: 2372, dtype: int64

In [62]:
y_test

Sales_ID
4049    1
4832    1
5124    0
6506    0
47      0
       ..
4192    1
4047    1
4731    0
2009    0
602     0
Length: 2372, dtype: int64

In [63]:
def confusion_matrix(t,y,labels):
    cm = pd.DataFrame(columns=labels,index=labels)
    d = {'Prediction': y, 'Actual': t}
    result = pd.DataFrame(data=d)
    
    cm[0][0] = len(result.loc[(result['Prediction']==0) & (result['Actual']==0)]) #TN 
    cm[0][1] = len(result.loc[(result['Prediction']==0) & (result['Actual']==1)]) #FN 
    cm[1][0] = len(result.loc[(result['Prediction']==1) & (result['Actual']==0)]) #FP
    cm[1][1] = len(result.loc[(result['Prediction']==1) & (result['Actual']==1)]) #TP
    
    return cm

In [64]:
cm = confusion_matrix(t_test,y_test,labels=[0,1])
cm

,0,1
0,1152,893
1,50,277


In [65]:
def evaluation(cm,positive_class=1):
    stats = {}
    if (positive_class==1):
        accuracy= (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][0]+cm[0][1]+cm[1][1])
        sensitivity= cm[1][1]/(cm[1][1]+cm[0][1])
        specificity= cm[0][0]/(cm[0][0]+cm[1][0])
        precision=cm[1][1]/(cm[1][1]+cm[1][0])
        F1= cm[1][1]/(cm[1][1]+0.5*(cm[1][0]+cm[0][1]))
        
    else:
        accuracy= (cm[1][1]+cm[0][0])/(cm[1][1]+cm[0][1]+cm[1][0]+cm[0][0])
        sensitivity= cm[0][0]/(cm[0][0]+cm[1][0])
        specificity= cm[1][1]/(cm[1][1]+cm[0][1])
        precision=cm[0][0]/(cm[0][0]+cm[0][1])
        F1= cm[0][0]/(cm[0][0]+0.5*(cm[0][1]+cm[1][0]))
        
    stats = {'accuracy':accuracy,'sensitivity/recall':sensitivity,'specificity':specificity,'precision':precision,'F1':F1}
    
    return stats

def evaluate(cm):
    stats = {}
    specific_stats = {}
    for x in y_test.unique():
        specific_stats[x] = evaluation(cm,positive_class=x)
        for y in specific_stats[x].keys():
            specific_stats[x][y] = specific_stats[x][y]*y_test.value_counts()[x]
    for x in specific_stats[0].keys():
        stats[x] = (specific_stats[0][x] + specific_stats[1][x])/sum(y_test.value_counts())
    return stats

In [66]:
evaluate(cm)

{'accuracy': 0.6024451939291737,
 'sensitivity/recall': 0.7032958633418384,
 'specificity': 0.7071241212554864,
 'precision': 0.6024451939291737,
 'F1': 0.5421158577436122}

In [71]:
def importance(X,t,seeds):
    importances = pd.Series(np.zeros((X.shape[1],)),index=X.columns)
    initial_importances = {}
    result = {}
    w = {}
    X_test = {}
    t_test = {}
    y_test = {}
    for seed in seeds:
        w[seed],X_test[seed],t_test[seed],results1 = train(X,t,seed=seed)
        y_test[seed] = predict(w[seed],X_test[seed])
        result[seed] = [math.sqrt(x**2) for x in w[seed]]
        initial_importances[seed] = [x/max(result[seed])for x in result[seed]]
    
    for seed in seeds:
        importances = list(map(add, importances, initial_importances[seed]))
    importances = [x / len(seeds) for x in importances]
    importances = pd.Series(importances,index=X.columns)    
    return importances

def feature_importance(X, y):
    specfic_importances = {}
    for index in X.index:
        for feature in y:
            if index == feature:
                specfic_importances[index] = X[index]
                break
    return specfic_importances  

In [68]:
seeds = [0,1,2,3,4,5]
importances = importance(predictors,target,seeds)

In [72]:
fi = feature_importance(importances, features)
{k: v for k, v in sorted(fi.items(), key=lambda item: item[1], reverse=True)}

{'selling_price': 1.0,
 'max_power': 0.8762117403660303,
 'year': 0.4050728661535725,
 'km_driven': 0.355586321069064,
 'engine': 0.3366540713862097,
 'mileage': 0.2933291608973722}